# Import Libraries

In [147]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.utils import resample, shuffle
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay, f1_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Create Features 

In [150]:
def continuous_data_features(df):
    indicators = pd.DataFrame(index=df.index)

    close = df["Close"]
    high = df["High"]
    low = df["Low"]
    vol = df["Volume"]

    # Simple Moving Average 10 day
    indicators["SMA10"] = close.rolling(window=10).mean()

    # Weighted Moving Average 10 day
    weights = np.arange(1, 11)
    indicators["WMA10"] = close.rolling(10).apply(lambda x: np.dot(x, weights)/weights.sum(), raw=True)

    # Momentum 10 day
    indicators["MOM10"] = close - close.shift(10)

    # Stochastic Oscillator %K 14 day
    indicators["STOCHK"] = 100 * (close - low.rolling(14).min()) / (high.rolling(14).max() - low.rolling(14).min())

    # Stochastic Oscillator %D 3 day SMA of %K
    indicators["STOCHD"] = indicators["STOCHK"].rolling(3).mean()

    # Relative Strength Index 14 day 
    delta = close.diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    rs = up.rolling(14).mean() / down.rolling(14).mean()
    indicators["RSI14"] = 100 - (100 / (1 + rs))

    # MACD 12-day EMA - 26 day EMA
    ema12 = close.ewm(span=12, adjust=False).mean()
    ema26 = close.ewm(span=26, adjust=False).mean()
    indicators["MACD"] = ema12 - ema26

    # Williams %R 14 day
    highest14 = high.rolling(14).max()
    lowest14 = low.rolling(14).min()
    indicators["WILLR"] = -100 * (highest14 - close) / (highest14 - lowest14)

    # 9. Accumulation/Distribution Oscillator (ADOSC)
    clv = np.where((high - low) == 0, 0, ((close - low) - (high - close)) / (high - low))
    adl = (clv * vol).cumsum()

    ema3_adl = adl.ewm(span=3, adjust=False).mean()
    ema10_adl = adl.ewm(span=10, adjust=False).mean()

    adosc = ema3_adl - ema10_adl

    # Scale ADOSC to 0–100 like in papers
    min_val = adosc.min()
    max_val = adosc.max()
    indicators["ADOSC"] = 100 * (adosc - min_val) / (max_val - min_val)


    # 10. Commodity Channel Index 20 day
    tp = (high + low + close) / 3
    sma = tp.rolling(20).mean()
    mad = (tp - sma).abs().rolling(20).mean()
    indicators["CCI20"] = (tp - sma) / (0.015 * mad)

    indicators["Close"] = close

    return indicators

In [152]:
def trend_deterministic_data(indicators, df):
    trend = pd.DataFrame(index=indicators.index)

    close = indicators["Close"]
    high = df["High"]
    low = df["Low"]
    vol = df["Volume"]

    #SMA
    trend["SMA10_T"] = np.where(close > indicators["SMA10"], 1, -1)

    #WMA
    trend["WMA10_T"] = np.where(close > indicators["WMA10"], 1, -1)

    #Momentum
    trend["MOM10_T"] = np.where(indicators["MOM10"] > 0, 1, -1)

    # Sophisticated Oscillator trends
    trend["STOCHK_T"] = np.where(indicators["STOCHK"] > indicators["STOCHK"].shift(1), 1, -1)
    trend["STOCHD_T"] = np.where(indicators["STOCHD"] > indicators["STOCHD"].shift(1), 1, -1)

    #RSI trend
    rsi = indicators["RSI14"]
    trend["RSI14_T"] = np.select([rsi > 70, rsi < 30, rsi > rsi.shift(1)],
                                 [-1, 1, 1], default=-1)
    #MACD trend
    trend["MACD_T"] = np.where(indicators["MACD"] > indicators["MACD"].shift(1), 1, -1)

    #Williams %R trend
    trend["WILLR_T"] = np.where(indicators["WILLR"] > indicators["WILLR"].shift(1), 1, -1)

    #ADOSC trend
    trend["ADOSC_T"] = np.where(indicators["ADOSC"] > indicators["ADOSC"].shift(1), 1, -1)

    #CCI trend
    cci = indicators["CCI20"]
    trend["CCI20_T"] = np.select([cci > 200, cci < -200, cci > cci.shift(1)],
                                 [-1, 1, 1], default=-1)
    return trend
    

## Loop over tickers

In [155]:
tickers = ["MSFT", "AMZN", "^GSPC", "^DJI"]
results = []

## Create Continuous Naive Bayes Model
Gaussian 

In [158]:
def continuous_nb(X, y):
    results = []

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.5, random_state=42, shuffle=True
    )

    # Scale features to (-1, 1)
    scaler = MinMaxScaler(feature_range=(-1, 1))
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    nb = GaussianNB()
    nb.fit(X_train_scaled, y_train)
    y_pred = nb.predict(X_test_scaled)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    results.append((acc, f1))

    df_results = pd.DataFrame(results, columns=["Accuracy", "F1"])
    return df_results


## Trend Naive Bayes

In [168]:
def trend_nb(X, y):
    results = []

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.5, random_state=42, shuffle=True
    )

    nb = BernoulliNB()
    nb.fit(X_train, y_train)
    y_pred = nb.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    results.append((acc, f1))

    df_results = pd.DataFrame(results, columns=["Accuracy", "F1"])
    return df_results

## Evaluate Models 

In [171]:
for ticker in tickers:
    print(f"Results for {ticker}")
    df = yf.download(ticker, start="2015-01-01", end="2024-12-31", auto_adjust=True)
    df.dropna(inplace=True)

    df["Target"] = np.where(df["Close"] > df["Close"].shift(1), 1, -1)
    df.dropna(inplace=True)

    y = df["Target"]
    X_cont_full = continuous_data_features(df)
    X_trend = trend_deterministic_data(X_cont_full, df)
    X_cont = X_cont_full.drop(columns=["Close"])

    cont_aligned = pd.concat([X_cont, y], axis=1)
    cont_aligned.dropna(inplace=True)
    X_cont_clean = cont_aligned.drop(columns=['Target'])
    y_cont_clean = cont_aligned['Target']

    trend_aligned = pd.concat([X_trend, y], axis=1)
    trend_aligned.dropna(inplace=True)
    X_trend_clean = trend_aligned.drop(columns=['Target'])
    y_trend_clean = trend_aligned['Target']

    cont_results = continuous_nb(X_cont_clean, y_cont_clean)
    print("Naive Bayes Continuous Results")
    print(cont_results[["Accuracy", "F1"]].to_string(index=False))

    trend_results = trend_nb(X_trend_clean, y_trend_clean)
    print("Naive Bayes Trend Results")
    print(trend_results[["Accuracy", "F1"]].to_string(index=False))
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Results for MSFT
Naive Bayes Continuous Results
 Accuracy       F1
 0.634383 0.683438
Naive Bayes Trend Results
 Accuracy       F1
 0.919714 0.924458
Results for AMZN
Naive Bayes Continuous Results
 Accuracy    F1
 0.618241 0.656
Naive Bayes Trend Results
 Accuracy       F1
 0.915739 0.920659
Results for ^GSPC
Naive Bayes Continuous Results
 Accuracy       F1
 0.625504 0.681756
Naive Bayes Trend Results
 Accuracy       F1
 0.926073 0.930649
Results for ^DJI
Naive Bayes Continuous Results
 Accuracy      F1
 0.630347 0.68544
Naive Bayes Trend Results
 Accuracy       F1
  0.90779 0.914328
